In [0]:
!pip install tweet-preprocessor
!pip install autocorrect

In [0]:
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd

#UserInfo.tsv
train=pd.read_csv('/content/gdrive/My Drive/task3_validation.tsv',delimiter='\t',encoding='utf-8')
#test=pd.read_csv('/content/gdrive/My Drive/task3_validation.tsv',delimiter='\t',encoding='utf-8')

In [0]:
train = train.filter(['tweet_id','begin','end','type','extraction','drug','tweet','meddra_code','meddra_term'])

In [0]:
from autocorrect import Speller
spell = Speller(lang='en')

print(spell("sicksicksick"))

sicksicksick


In [0]:
isnan= 0
for i in range(len(train)):
  if train.iloc[i]['type'] == "ADR":
    isnan+=1
print(isnan)


365


In [0]:
b = TextBlob("halfasleep dasd")
text=str(b.correct())
print(text)

NameError: ignored

In [0]:
len(set(train['tweet'].to_list()))

422

In [0]:
med_list_from_train=[]
for x in list(set(train['drug'].to_list())):
  med_list_from_train.append(str(x).lower())
#set(med_list_from_train)
import pickle
with open('/content/gdrive/My Drive/med_list_from_train.pkl', 'wb') as f:
  pickle.dump(med_list_from_train, f)

In [0]:
med_list_from_train

['nan',
 'rivaroxaban',
 'trazodone',
 'valproate',
 'effexor',
 'baclofen',
 'enbrel',
 'viibryd',
 'quetiapine',
 'olanzapine',
 'humira',
 'namenda',
 'zyprexa',
 'prozac',
 'fluoxetine',
 'avelox',
 'seroquel',
 'pregabalin',
 'zoloft',
 'venlafaxine',
 'cipro',
 'robitussin',
 'ofloxacin',
 'leflunomide',
 'lamictal',
 'duloxetine',
 'lexapro',
 'metoprolol',
 'cymbalta',
 'lyrica',
 'topomax',
 'melatonin',
 'lozenge',
 'pristiq',
 'tramadol',
 'prozac',
 'citalopram',
 'tysabri',
 'nicotine',
 'cipro',
 'ritalin',
 'paxil',
 'vanlafaxine',
 'vyvanse',
 'geodon',
 'lamotrigine',
 'memantine',
 'levaquin']

In [0]:
import pickle
with open('/content/gdrive/My Drive/Task2_data/rxlist_meds.pkl', 'rb') as f:
  drug_list = pickle.load(f)
drug_list.append("zoloft")

In [0]:
from textblob import Word, TextBlob
import string
from nltk.corpus import stopwords
from nltk import SnowballStemmer as Stemmer
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# text = u'This dog \U0001f602'
# print(text) # with emoji

emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
############################################
def correct(text):
  s=""
  for x in text.split():
    if(x=="rt"):
      continue
    u=Word(x)
    s=s+u.lemmatize()+" "
  return s.strip()

##################################################
def numberconv(Number):
  if(Number>20):
    Number=Number-Number%10
  if(Number>100):
     Number=Number-Number%100
  import inflect
  p = inflect.engine()
  return(p.number_to_words(Number))
#################################################

def lemma(text):
  s=""
  for x in text.split():
    if(x=="rt"):
      continue
    elif(x=="pounds" or x=="kg" or x=="lb" or x=="lbs" or x=="gm" or x=="gm" or x=="kgs" or x=="grams" or x=="kilograms"):
      s=s+"amount"+" "
      continue
    elif(x.isnumeric()):
      s=s+numberconv(int(x))+" "
    # elif(x in drug_list):
    #   s=s+"drug"+" "
    else:
      u=Word(x)
      s=s+u.lemmatize()+" "
  return s.strip()
#print(emoji_pattern.sub(r'', text)) # no emoji


def remove_emoji(text):
  return emoji_pattern.sub(r'', text)


def process(text):
  text = p.clean(text)
  text= re.sub(r'\s([@][\w_-]+)', '', text, flags=re.MULTILINE)
  text=remove_emoji(text)
  x=text.lower()
  x = ''.join([t for t in x if t not in string.punctuation])
  #b = TextBlob(x)
  #text=str(b.correct())
  text=lemma(x)
  return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
tweets=[]
counter=0
for tweet in train['tweet'].to_list():
  #print(counter)
  tweets.append(process(tweet))
  #counter=counter+1

In [0]:
extracted=[]
for tweet in train['extraction'].to_list():
  if(str(tweet)=="nan"):
    extracted.append("")
  else:
    extracted.append(process(tweet))

In [0]:
import math, spacy
from spacy.gold import biluo_tags_from_offsets
from spacy.tokenizer import Tokenizer

nlp = spacy.load('en_core_web_sm')
nlp.tokenizer = Tokenizer(nlp.vocab)
def label_making(sentence, begin, end):
  doc = nlp(sentence)
  if(math.isnan(begin)):
    entities = []
  else:
    # print(int(begin[0])-1)
    # print(int(end[0])-1)
    entities = [(int(begin), int(end), "ADR")]
  tags = biluo_tags_from_offsets(doc, entities)
  return tags,doc

In [0]:
label_making("do you have any medication allergies ",27,36)

(['O', 'O', 'O', 'O', 'O', 'U-ADR'], do you have any medication allergies )

In [0]:
def processextrac(sent,extraction):
  #print(sent)
  start_index=sent.find(extraction)
  end_index=start_index+len(extraction)
  tags,doc=label_making(sent,start_index,end_index)
  tokens=[]
  for token in doc:
    tokens.append(str(token))
  return tokens, tags
  # tokens=[]
  # dict_word=extraction.split()
  # tags=[]
  # counter=0
  # for token in sent.split():
  #   tokens.append(token)
  #   if(token in dict_word and counter<len(dict_word)):
  #     # if(counter==0):
  #     #   tags.append("B-ADR")
  #     # elif(counter>0):
  #     #   tags.append("I-ADR")
  #     tags.append("I-ADR")
  #     counter=counter+1
  #   else:
  #     tags.append("O")
  # if(counter==1):
  #   index = tags.index("I-ADR")
  #   tags[index]="U-ADR"
  # elif(counter>1):
  #   index = tags.index("I-ADR")
  #   tags[index]="B-ADR"
  #   res = max(idx for idx, val in enumerate(tags)if val == "I-ADR")
  #   tags[res]="L-ADR"
  # return tokens, tags

In [0]:
xtags=[]
xtokens=[]
for i in range(len(tweets)):
  tags_per_tweet=[]
  tokens_per_tweet=[]
  if(len(extracted[i])==0):
    for token in tweets[i].split():
      tokens_per_tweet.append(token)
      tags_per_tweet.append('O')
  else:
    tokens_per_tweet,tags_per_tweet=processextrac(tweets[i],extracted[i])
  xtags.append(tags_per_tweet)
  xtokens.append(tokens_per_tweet)

In [0]:
for row in xtags:
  if(row.count('-')==0):
    continue
  if(row.count('-')==1):
    row[row.index("-")]='U-ADR'
  elif(row.count('-')==2):
    row[row.index("-")]='B-ADR'
    row[row.index("-")]='L-ADR'
  else:
    row[row.index("-")]='B-ADR'
    while(row.count('-')>1):
      row[row.index("-")]='I-ADR'
    row[row.index("-")]='L-ADR'

In [0]:
for i in range(len(xtokens)):
  if len(xtokens[i])!=len(xtags[i]):
    print("Problem at ",i)

In [0]:
extracted

['allergy',
 'hurt your liver',
 'ad',
 'focus',
 'died',
 'died',
 'tendon damage',
 '',
 '',
 'dream',
 'tendon rupture',
 '',
 '',
 'withdrawal',
 'emotional mess',
 'not remember',
 'memory lapse',
 'nap',
 'psychedelic',
 'mashed',
 '',
 '',
 '',
 'space cadet',
 'illness worse',
 'cant eat',
 '',
 'allergic',
 'withdrawal',
 'hungry',
 '',
 'nerve',
 'muscle spasm',
 '',
 'joint pain',
 'painful reaction',
 'fistula',
 'achy',
 'fatigue',
 'fatigue',
 'achy',
 'no immune',
 'nerve damage',
 'm like',
 'seizure',
 'pain',
 'weakness',
 'rash',
 'hair is definitely falling out',
 'sick',
 'depressed',
 'day of hell',
 'clusterheads',
 'addictive',
 'allergic',
 'cant think',
 'see straight',
 '',
 'starvation',
 'increase hunger',
 'dream',
 'fat',
 'hand shake',
 'stopped working',
 'gaining',
 'gain like fifty amount',
 'getting out of bed is haaard',
 'creativity wa blocked',
 'wd',
 'drowsy',
 'haze',
 'brain zap',
 'genitals arent numb',
 'detox',
 'facial spasm',
 'eye',
 'aw

In [0]:
print(len(xtokens))
print(len(train))
train

560
560


,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
555,331289838856835072,NaN,NaN,NaN,NaN,NaN,"okay trazodone, work your magic.",NaN,NaN
556,332907977700937731,NaN,NaN,NaN,NaN,NaN,"tomorrow, sending mummers her mother's day pos...",NaN,NaN
557,333123450254274560,NaN,NaN,NaN,NaN,NaN,@kyleecakes4 i was given like 5mg of abilify a...,NaN,NaN
558,341950988455927808,NaN,NaN,NaN,NaN,NaN,"stand down, tweeps! stand down! i did it wrong...",NaN,NaN


In [0]:
final_train=[]
new_final_train = []


for i in range(len(xtokens)):
  temp = []
  begin = train.iloc[i]["begin"]
  end = train.iloc[i]["end"]
  extraction = train.iloc[i]["extraction"]
  tweet_id = train.iloc[i]["tweet_id"]
  temp = [begin,end,extraction,tweet_id,train.iloc[i]['tweet'],xtokens[i],xtags[i],train.iloc[i]['meddra_code'],train.iloc[i]['meddra_term']]
  new_final_train.append(temp)



  tup=(xtokens[i],xtags[i])
  final_train.append(tup)
  print(i,"---->",final_train[i])

0 ----> (['do', 'you', 'have', 'any', 'medication', 'allergy', 'asthma', 'me', 'pt', 'no', 'wait', 'avelox', 'thats', 'it', 'so', 'no', 'other', 'allergy', 'right', 'cont'], ['O', 'O', 'O', 'O', 'O', 'U-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
1 ----> (['if', 'avelox', 'ha', 'hurt', 'your', 'liver', 'avoid', 'tylenol', 'always', 'a', 'it', 'further', 'damage', 'liver', 'eat', 'grapefruit', 'unless', 'taking', 'cardiac', 'drug'], ['O', 'O', 'O', 'B-ADR', 'I-ADR', 'L-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
2 ----> (['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', 'ad', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'today', 'about', 'thirty', 'second'], ['O', 'O', 'O', 'O', 'O', 'U-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
3 ----> (['apparently', 'baclofen', 'greatly', 'exacerbates', 'the', 'ad', 'part', 'of', 'my', 'adhd', 'average', 'length', 'of', 'focus', 'tod

In [0]:
new_train = pd.DataFrame(new_final_train, columns=['begin','end','extraction','tweet_id','tweet','tokens','tags','meddra_code','meddra_term'],)

In [0]:
new_train.dtypes

begin          float64
end            float64
extraction      object
tweet_id         int64
tweet           object
tokens          object
tags            object
meddra_code    float64
meddra_term     object
dtype: object

In [0]:
code_comb = []
term_comb = []
extr_comb = []
for i in range(len(new_train)):
  temp_code = []
  temp_term = []
  temp_extr =[]
  if str(new_train.iloc[i]['extraction']) != "nan":
    # continue
    idd = new_train.iloc[i]['tweet_id']
    
    while(idd == new_train.iloc[i]['tweet_id']):
      temp_code.append([new_train.iloc[i]['meddra_code']])
      temp_term.append([new_train.iloc[i]['meddra_term']])
      temp_extr.append([new_train.iloc[i]['extraction']])
      i+=1
  code_comb.append(temp_code)
  term_comb.append(temp_term)
  extr_comb.append(temp_extr)



In [0]:
code_comb

[[[10013661.0]],
 [[10024668.0]],
 [[10003731.0], [10003738.0]],
 [[10003738.0]],
 [[10011906.0]],
 [[10011906.0]],
 [[10043242.0]],
 [],
 [],
 [[10004969.0]],
 [[10043248.0]],
 [],
 [],
 [[10048010.0]],
 [[10049119.0]],
 [[10027176.0], [10027174.0]],
 [[10027174.0]],
 [[10041349.0]],
 [[10070679.0]],
 [[10070679.0]],
 [],
 [],
 [],
 [[10041374.0]],
 [[10021345.0], [10003030.0]],
 [[10003030.0]],
 [],
 [[10013661.0]],
 [[10048010.0]],
 [[10016336.0]],
 [],
 [[10029177.0], [10028334.0]],
 [[10028334.0]],
 [],
 [[10023222.0]],
 [[10033371.0], [10016717.0]],
 [[10016717.0]],
 [[10000424.0], [10016256.0]],
 [[10016256.0]],
 [[10016256.0], [10000424.0]],
 [[10000424.0]],
 [[10021425.0]],
 [[10029177.0], [10028055.0]],
 [[10028055.0]],
 [[10039906.0], [10033371.0], [10047862.0]],
 [[10033371.0], [10047862.0]],
 [[10047862.0]],
 [[10062948.0]],
 [[10019045.0]],
 [[10016365.0]],
 [[10012374.0]],
 [[10016370.0], [10059133.0], [10012336.0]],
 [[10059133.0], [10012336.0]],
 [[10012336.0]],
 [[100

In [0]:
new_train_gb = new_train.dropna(subset=['extraction'])

In [0]:
new_train_gb = new_train.groupby('tweet_id').agg(lambda x: x.tolist())


In [0]:
new_train_gb

,begin,end,extraction,tweet,tokens,tags,meddra_code,meddra_term
tweet_id,,,,,,,,
326376825590779905,[nan],[nan],[nan],[do u know what medications are r for bipolar ...,"[[do, u, know, what, medication, are, r, for, ...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan]
326406322323066883,"[111.0, 45.0]","[117.0, 71.0]","[zombie, feel like crying but can't]",[thought of work is overwhelming me so much i ...,"[[thought, of, work, is, overwhelming, me, so,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016384.0, 10011469.0]","[felt like a zombie, crying]"
326455480891346947,"[108.0, 46.0]","[122.0, 55.0]","[hot &amp; cold, Lethargic]",[5hrs sleep last night but stayed awake today....,"[[5hrs, sleep, last, night, but, stayed, awake...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016335.0, 10024262.0]","[feeling hot and cold, lethargic]"
326456822007492608,[nan],[nan],[nan],[my urologist today said that trazodone can ca...,"[[my, urologist, today, said, that, trazodone,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan]
326575463835250689,[51.0],[56.0],[awake],[@rbhu7_82 if i don't take my quetiapine i can...,"[[if, i, dont, take, my, quetiapine, i, can, s...","[[O, O, O, O, O, O, O, O, O, U-ADR, O, O]]",[10041014.0],[sleepiness]
...,...,...,...,...,...,...,...,...
353707236402536449,[nan],[nan],[nan],[mark munoz plans on taking paxil inbetween ro...,"[[mark, munoz, plan, on, taking, paxil, inbetw...","[[O, O, O, O, O, O, O, O, O]]",[nan],[nan]
353837855770030081,[nan],[nan],[nan],[@doctormcpa @sarahrnay am i reading this corr...,"[[am, i, reading, this, correctlyare, you, pre...","[[O, O, O, O, O, O, O, O, O, O, O]]",[nan],[nan]
354228220431843328,[94.0],[100.0],[mashed],[@santypas @scrabalisios i'm taking 700mg preg...,"[[im, taking, 700mg, pregablin, and, sixty, mg...","[[O, O, O, O, O, O, O, O, O, O, O, O, U-ADR]]",[10070679.0],[feeling stoned]


In [0]:
for i in range(len(new_train_gb)):
  print(i,"---",new_train_gb.iloc[i])

0 --- begin                                                      [nan]
end                                                        [nan]
extraction                                                 [nan]
tweet          [do u know what medications are r for bipolar ...
tokens         [[do, u, know, what, medication, are, r, for, ...
tags           [[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...
meddra_code                                                [nan]
Name: 326376825590779905, dtype: object
1 --- begin                                              [111.0, 45.0]
end                                                [117.0, 71.0]
extraction                  [zombie, feel like crying but can't]
tweet          [thought of work is overwhelming me so much i ...
tokens         [[thought, of, work, is, overwhelming, me, so,...
tags           [[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...
meddra_code                             [10016384.0, 10011469.0]
Name: 326406322323066883, dtype: objec

In [0]:
new_intervals[2]

[[46.0, 55.0], [108.0, 122.0]]

In [0]:
print(new_train.tweet_id == 351677046755303424)

0      False
1      False
2      False
3      False
4      False
       ...  
555    False
556    False
557    False
558    False
559    False
Name: tweet_id, Length: 560, dtype: bool


In [0]:
def newmerge(er):
  temp_er = er.sort(key=lambda x:x[0])
  return er, True
  # temp = []
  # ret = True
  # for e in er:
  #   if not temp or temp[-1][-1]<e[0]:
  #     temp.append(e)
  #   else:
  #     temp[-1][-1] = max(temp[-1][-1], e[-1])
  #     ret = False
  # return temp,ret

In [0]:
count = 0
del_tweets = []
new_intervals = []
for i in range(len(new_train_gb)):
  er = []
  for j in range(len(new_train_gb.iloc[i]['begin'])):
    er.append([new_train_gb.iloc[i]['begin'][j],new_train_gb.iloc[i]['end'][j]])
  interval, ret = newmerge(er)
  new_intervals.append(interval)
  if not ret:
    del_tweets.append(new_train_gb.iloc[i])
    print(new_train_gb.iloc[i])
    count+=1
print(count)


0


In [0]:
new_train_gb['new_intervals'] = new_intervals

In [0]:
import pickle
with open('/content/gdrive/My Drive/nlp_task2/final_task2_onlytesting_nopred.pkl', 'wb') as f:
  pickle.dump(new_train_gb, f)

In [0]:
new_train_gb.to_csv('/content/gdrive/My Drive/nlp_task2/newtask2_dataset_with_sintervals.csv',index=False,header=True)

In [0]:
del_list = [351677046755303424,351442731454828544,350859685227798531,349455785308327936,349382072810483712,348662789964972032,348355774642266112,348136954900127744,\
            347824349572644865,347414282138431488,346054053182193665,345762127270793217,345280161941430272,344733595103002624,\
            343885385077768193,343130110569873409,342843798554046464,342730668704415745,342479732811644929]
print(type(new_train.iloc[0]['tweet_id']))
del_tweet = {}
for i in range(len(new_train)):
  if new_train.iloc[i]['tweet_id'] in del_list:
    key = new_train.iloc[i]['tweet_id']
    if str(key) not in del_tweet:
      del_tweet[str(key)] = [[new_train.iloc[i]['tokens'],new_train.iloc[i]['tags']]]
    else:
      del_tweet[str(key)].append([new_train.iloc[i]['tokens'],new_train.iloc[i]['tags']])

print(len(del_tweet))


  



<class 'numpy.int64'>
0


In [0]:
import json

j = json.dumps(del_tweet)
print(j)
f = open('/content/gdrive/My Drive/nlp_task2/delete_list.json', "w")
f.write(j)
f.close
# with open('/content/gdrive/My Drive/nlp_task2/delete_list.txt', 'w') as fp:
  # for 

{"342479732811644929": [[["id", "gladly", "trade", "the", "mtx", "headache", "for", "humira", "but", "after", "five", "month", "i", "still", "dont", "have", "the", "prescription", "i", "function", "through", "vitamin", "amp", "motrin"], ["O", "O", "O", "O", "O", "U-ADR", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]], [["id", "gladly", "trade", "the", "mtx", "headache", "for", "humira", "but", "after", "five", "month", "i", "still", "dont", "have", "the", "prescription", "i", "function", "through", "vitamin", "amp", "motrin"], ["O", "O", "O", "O", "O", "U-ADR", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]]], "342730668704415745": [[["what", "problem", "did", "you", "have", "with", "humira", "if", "you", "dont", "mind", "me", "asking", "i", "tried", "infliximab", "it", "destroyed", "my", "liver"], ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-ADR", "I-ADR",

<function TextIOWrapper.close>

In [0]:
# def get_training_list(train, xtokens):
#   new_final_train = []
#   for i in range(len(xtokens)):
#     temp = []
#     begin = train.iloc[i]["begin"]
#     end = train.iloc[i]["end"]
#     extraction = train.iloc[i]["extraction"]
#     tweet_id = train.iloc[i]["tweet_id"]
#     temp = [begin,end,extraction,tweet_id,train.iloc[i]['tweet'],xtokens[i],xtags[i],]
#     new_final_train.append(temp)
#   return new_final_train

del_train = []

for i in range(len(new_train)):
  if new_train.iloc[i]['tweet_id'] not in del_list:
    temp = []
    begin = new_train.iloc[i]["begin"]
    end = new_train.iloc[i]["end"]
    extraction = new_train.iloc[i]["extraction"]
    tweet_id = new_train.iloc[i]["tweet_id"]
    temp = [begin,end,extraction,tweet_id,train.iloc[i]['tweet'],new_train.iloc[i]['tokens'],new_train.iloc[i]['tags'],new_train.iloc[i]['meddra_code']]
    del_train.append(temp)


In [0]:
len(del_train)

560

In [0]:
del_train_pd = pd.DataFrame(del_train, columns=['begin','end','extraction','tweet_id','tweet','tokens','tags','meddra_code'])

In [0]:
del_train_pd.to_csv('/content/gdrive/My Drive/nlp_task2/del_train_pd.csv',index=False,header=True)

In [0]:
del_train_pdgb = del_train_pd.groupby('tweet_id').agg(lambda x: x.tolist())

In [0]:
print(len(del_train_pdgb))

428


In [0]:
del_train_pd.to_csv('/content/gdrive/My Drive/nlp_task2/deleted_tweets_gb.csv',index=False,header=True)
#just to check
# temp_df =pd.DataFrame( columns=['begin','end','extraction','tweet_id','tweet','tokens','tags'])
# for i in range(len(del_train_pd)):
#   if del_train_pd.iloc[i]['tweet_id'] == 358354458083729408:
#     print("inside")
#     temp_df = temp_df.append(del_train_pd.iloc[i])

# temp_df_gb = temp_df.groupby('tweet_id').agg(lambda x: x.tolist())
# temp_df_gb

In [0]:
deleted_tweets_gb = pd.read_csv('/content/gdrive/My Drive/nlp_task2/deleted_tweets_gb.csv', encoding='utf-8')

In [0]:
del_train_pd
# for i in range(10):
#   print(del_train_pgdb.iloc[i]['tweet_id'])

for i in range(len(del_train_pd)):
  


,begin,end,extraction,tweet_id,tweet,tokens,tags,meddra_code
0,28.0,37.0,allergies,332317478170546176,"do you have any medication allergies? ""asthma!...","[do, you, have, any, medication, allergy, asth...","[O, O, O, O, O, U-ADR, O, O, O, O, O, O, O, O,...",10013661.0
1,31.0,46.0,HURT YOUR Liver,347806215776116737,"@ashleylvivian if #avelox has hurt your liver,...","[if, avelox, ha, hurt, your, liver, avoid, tyl...","[O, O, O, B-ADR, I-ADR, L-ADR, O, O, O, O, O, ...",10024668.0
2,48.0,50.0,AD,350336129817509888,"apparently, baclofen greatly exacerbates the ""...","[apparently, baclofen, greatly, exacerbates, t...","[O, O, O, O, O, U-ADR, O, O, O, O, O, O, O, O,...",10003731.0
3,88.0,93.0,focus,350336129817509888,"apparently, baclofen greatly exacerbates the ""...","[apparently, baclofen, greatly, exacerbates, t...","[O, O, O, O, O, O, O, O, O, O, O, O, O, U-ADR,...",10003738.0
4,11.0,15.0,died,332540699692130304,pt of mine died from cipro rt @ciproispoison: ...,"[pt, of, mine, died, from, cipro, if, only, mo...","[O, O, O, U-ADR, O, O, O, O, O, O, O, O, O, O,...",10011906.0
...,...,...,...,...,...,...,...,...
555,NaN,NaN,NaN,331289838856835072,"okay trazodone, work your magic.","[okay, trazodone, work, your, magic]","[O, O, O, O, O]",NaN
556,NaN,NaN,NaN,332907977700937731,"tomorrow, sending mummers her mother's day pos...","[tomorrow, sending, mummer, her, mother, day, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O]",NaN
557,NaN,NaN,NaN,333123450254274560,@kyleecakes4 i was given like 5mg of abilify a...,"[i, wa, given, like, 5mg, of, abilify, and, i,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN
558,NaN,NaN,NaN,341950988455927808,"stand down, tweeps! stand down! i did it wrong...","[stand, down, tweeps, stand, down, i, did, it,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",NaN


In [0]:
del_train_pdgb
# isnan =0
# for i in range(len(del_train_pdgb)):
#   if math.isnan(del_train_pdgb.iloc[i]['begin'][0]):
#     isnan+=1
# print(isnan)


,begin,end,extraction,tweet,tokens,tags,meddra_code
tweet_id,,,,,,,
326376825590779905,[nan],[nan],[nan],[do u know what medications are r for bipolar ...,"[[do, u, know, what, medication, are, r, for, ...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan]
326406322323066883,"[111.0, 45.0]","[117.0, 71.0]","[zombie, feel like crying but can't]",[thought of work is overwhelming me so much i ...,"[[thought, of, work, is, overwhelming, me, so,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016384.0, 10011469.0]"
326455480891346947,"[108.0, 46.0]","[122.0, 55.0]","[hot &amp; cold, Lethargic]",[5hrs sleep last night but stayed awake today....,"[[5hrs, sleep, last, night, but, stayed, awake...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016335.0, 10024262.0]"
326456822007492608,[nan],[nan],[nan],[my urologist today said that trazodone can ca...,"[[my, urologist, today, said, that, trazodone,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan]
326575463835250689,[51.0],[56.0],[awake],[@rbhu7_82 if i don't take my quetiapine i can...,"[[if, i, dont, take, my, quetiapine, i, can, s...","[[O, O, O, O, O, O, O, O, O, U-ADR, O, O]]",[10041014.0]
...,...,...,...,...,...,...,...
353707236402536449,[nan],[nan],[nan],[mark munoz plans on taking paxil inbetween ro...,"[[mark, munoz, plan, on, taking, paxil, inbetw...","[[O, O, O, O, O, O, O, O, O]]",[nan]
353837855770030081,[nan],[nan],[nan],[@doctormcpa @sarahrnay am i reading this corr...,"[[am, i, reading, this, correctlyare, you, pre...","[[O, O, O, O, O, O, O, O, O, O, O]]",[nan]
354228220431843328,[94.0],[100.0],[mashed],[@santypas @scrabalisios i'm taking 700mg preg...,"[[im, taking, 700mg, pregablin, and, sixty, mg...","[[O, O, O, O, O, O, O, O, O, O, O, O, U-ADR]]",[10070679.0]


In [0]:
med = 0
import math
for i in range(len(del_train_pdgb)):
  meddra_code = del_train_pdgb.iloc[i]['meddra_code']
  if math.isnan(meddra_code[0]):
    print(True)
  

print(med)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
0


In [0]:
ret_xtokens = []
ret_xtags = []
ret_meddra_code = []
for i in range(len(del_train_pdgb)):
  
  tags = del_train_pdgb.iloc[i]['tags']
  tokens = del_train_pdgb.iloc[i]['tokens']
  meddra_code = del_train_pdgb.iloc[i]['meddra_code']
  print(meddra_code)
  
  
  wd = len(tags)
  lg = len(tags[0])
  new_xtags = []
  if wd ==1:
    #direct append
    new_xtags = tags[0]
    ret_xtokens.append(new_xtags)
    ret_xtags.append(tokens[0])
    continue
  else:
    
    for j in range(lg):
      b = False
      for k in range(wd):
        if tags[k][j] == 'B-ADR' or tags[k][j] == 'I-ADR' or tags[k][j] == 'L-ADR'\
        or tags[k][j] == 'U-ADR':
          new_xtags.append(tags[k][j])
          b = True
          break
      if not b:
        new_xtags.append('O')
  ret_xtokens.append(new_xtags)
  ret_xtags.append(tokens[0])
  ret_meddra_code.append(meddra_code)


# print(temp_df.iloc[0]['tags'])
# print(temp_df.iloc[1]['tags'])

      
print(ret_meddra_code)



[nan]
[10016384.0, 10011469.0]
[10016335.0, 10024262.0]
[nan]
[10041014.0]
[10041017.0, 10042112.0]
[10020466.0, 10020466.0]
[10013663.0]
[10016356.0, 10043890.0, 10023614.0]
[nan]
[nan]
[10012359.0]
[10041017.0, 10062519.0]
[nan]
[10041349.0, 10043890.0, 10013573.0]
[10041349.0]
[10015667.0, 10041349.0]
[10028339.0, 10028339.0]
[nan]
[10020466.0]
[10024919.0, 10016364.0, 10024919.0, 10062519.0]
[10041005.0]
[10011865.0]
[nan]
[nan]
[nan]
[10015598.0]
[10041349.0]
[10013649.0, 10062519.0]
[nan]
[nan]
[nan]
[nan]
[10041374.0, 10070679.0]
[nan]
[10013573.0, 10013710.0]
[10041014.0]
[nan]
[10016370.0, 10041007.0]
[10041349.0]
[10026749.0]
[nan]
[10004969.0]
[10043169.0, 10062519.0, 10048010.0]
[10023222.0, 10016370.0]
[nan]
[10007216.0]
[10001125.0]
[nan]
[nan]
[10013754.0, 10016335.0, 10019211.0, 10016365.0, 10028813.0, 10040558.0, 10045198.0]
[10004969.0]
[10023222.0]
[10041349.0]
[nan]
[10043436.0, 10047508.0]
[10016365.0]
[10016384.0]
[10004969.0, 10048010.0, 10004969.0]
[10012336.0, 

In [0]:
medd = []

for i in range(len(del_train_pdgb)):
  med = del_train_pdgb.iloc[i]['meddra_code']
  medd.append(med)
print((medd))

[[nan], [10016384.0, 10011469.0], [10016335.0, 10024262.0], [nan], [10041014.0], [10041017.0, 10042112.0], [10020466.0, 10020466.0], [10013663.0], [10016356.0, 10043890.0, 10023614.0], [nan], [nan], [10012359.0], [10041017.0, 10062519.0], [nan], [10041349.0, 10043890.0, 10013573.0], [10041349.0], [10015667.0, 10041349.0], [10028339.0, 10028339.0], [nan], [10020466.0], [10024919.0, 10016364.0, 10024919.0, 10062519.0], [10041005.0], [10011865.0], [nan], [nan], [nan], [10015598.0], [10041349.0], [10013649.0, 10062519.0], [nan], [nan], [nan], [nan], [10041374.0, 10070679.0], [nan], [10013573.0, 10013710.0], [10041014.0], [nan], [10016370.0, 10041007.0], [10041349.0], [10026749.0], [nan], [10004969.0], [10043169.0, 10062519.0, 10048010.0], [10023222.0, 10016370.0], [nan], [10007216.0], [10001125.0], [nan], [nan], [10013754.0, 10016335.0, 10019211.0, 10016365.0, 10028813.0, 10040558.0, 10045198.0], [10004969.0], [10023222.0], [10041349.0], [nan], [10043436.0, 10047508.0], [10016365.0], [1001

In [0]:
# counter =0
# for i in range(len(del_train_pdgb)):
#   print(type(del_train_pdgb.iloc[i]['extraction']))
#   # if str(del_train_pdgb.iloc[i]['extraction']) != str.isn:
#   #   counter+=1 
# print(counter)
print(len(ret_meddra_code))


99


In [0]:
dump_tags_singular = []
for i in range(len(ret_xtags)):
  tup = (medd[i],ret_xtokens[i])
  dump_tags_singular.append(tup)
  


In [0]:
import pickle

with open('/content/gdrive/My Drive/nlp_task2/test_singular_labels_meddra.pkl', 'wb') as f:
  pickle.dump(dump_tags_singular, f)

In [0]:
new_train.to_csv('/content/gdrive/My Drive/nlp_task2/new_train.csv',index=False,header=True)

In [0]:
read_new  = pd.read_csv('/content/gdrive/My Drive/nlp_task2/new_train.csv')
read_new

,begin,end,extraction,tweet_id,tweet,tokens,tags
0,NaN,NaN,NaN,331187619096588288,@seefisch:oral drugs for pyelonephritis:ciprof...,"['oral', 'drug', 'for', 'pyelonephritisciprofl...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,NaN,NaN,NaN,332227554956161024,happy for wellbutrin; has similar effects as a...,"['happy', 'for', 'wellbutrin', 'ha', 'similar'...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,NaN,NaN,NaN,332448217490944000,@stilgarg i'm ok ty have an official diagnosis...,"['im', 'ok', 'ty', 'have', 'an', 'official', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,NaN,NaN,NaN,332977955754110976,i'm soo depressed cymbalta couldn't help me .,"['im', 'soo', 'depressed', 'cymbalta', 'couldn...","['O', 'O', 'O', 'O', 'O', 'O', 'O']"
4,NaN,NaN,NaN,333674203331051520,time for my daily afternoon relaxation ritual ...,"['time', 'for', 'my', 'daily', 'afternoon', 'r...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...,...,...,...
2241,NaN,NaN,NaN,347921687729299456,i will admit that most people would agree zypr...,"['i', 'will', 'admit', 'that', 'most', 'people...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2242,72.0,91.0,gain so much weight,348383854089871360,that zyprexa really makes your vocal chords ma...,"['that', 'zyprexa', 'really', 'make', 'your', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2243,NaN,NaN,NaN,348595096352075776,@incubator04 i'm hoping that i get on the zypr...,"['im', 'hoping', 'that', 'i', 'get', 'on', 'th...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2244,99.0,117.0,increase my weight,349181683293102080,"i'm so fine today. increasing zyprexa,my condi...","['im', 'so', 'fine', 'today', 'increasing', 'z...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [0]:
def merge(intervals):
  if len(intervals)==0:
      return []
  intervals = sorted(intervals,key = lambda x : x[0])
  current = intervals[0]
  result = []
  for i in range(len(intervals)):
      if current[0]<=intervals[i][0]<=current[1]:
          current[1] = max(current[1],intervals[i][1])
          continue
      else:
          result.append(current)
          current = intervals[i]
  result.append(current)
  return result
  

In [0]:
new_extraction = []
beg_arr = []
end_arr = []
for i in range(len(new_train_gb)):
  er = []
  for j in range(len(new_train_gb.iloc[i]['begin'])):
    # if str(new_train_gb.iloc[i]['begin'][j]) == "nan":
    #   beg_arr.append(new_train_gb.iloc[i]['begin'][j])
    #   end_arr.append(new_train_gb.iloc[i]['begin'][j])
    #   new_extraction.append([new_train_gb.iloc[i]['begin'][j],new_train_gb.iloc[i]['begin'][j],"ADR"])
    #   # new_extraction.append(new_train_gb.iloc[i]['begin'])
    #   continue
    er.append([new_train_gb.iloc[i]['begin'][j],new_train_gb.iloc[i]['end'][j]])
  # if len(er)!= 0:
  ret = merge(er)
  print(ret)
  for r in ret:
    beg_arr.append(r[0])
    end_arr.append(r[1])
    new_extraction.append([int(r[0]),int(r[1]),"ADR"])
  # else:
  #   print("empty")


    
    


[[32.0, 43.0]]
[[67.0, 73.0]]
[[8.0, 26.0]]
[[40.0, 53.0]]
[[16.0, 27.0], [38.0, 47.0], [121.0, 131.0]]
[[132.0, 138.0]]
[[2.0, 5.0]]
[[31.0, 52.0]]
[[80.0, 84.0]]
[[36.0, 42.0]]
[[47.0, 62.0]]
[[76.0, 93.0]]
[[32.0, 46.0]]
[[36.0, 47.0]]
[[77.0, 115.0]]
[[52.0, 67.0], [72.0, 83.0]]
[[117.0, 131.0]]
[[91.0, 97.0]]
[[64.0, 82.0]]
[[57.0, 62.0]]
[[42.0, 62.0], [121.0, 137.0]]
[[42.0, 68.0]]
[[48.0, 58.0], [66.0, 91.0]]
[[24.0, 32.0]]
[[64.0, 79.0]]
[[86.0, 94.0]]
[[117.0, 128.0]]
[[59.0, 69.0]]
[[21.0, 32.0]]
[[96.0, 123.0]]
[[42.0, 54.0], [59.0, 69.0]]
[[34.0, 43.0], [48.0, 54.0]]
[[68.0, 74.0]]
[[43.0, 48.0]]
[[38.0, 47.0]]
[[47.0, 53.0], [58.0, 66.0]]
[[20.0, 30.0]]
[[66.0, 75.0]]
[[33.0, 48.0]]
[[23.0, 29.0]]
[[129.0, 141.0]]
[[87.0, 107.0]]
[[111.0, 115.0]]
[[62.0, 66.0]]
[[94.0, 101.0]]
[[76.0, 81.0], [83.0, 92.0], [102.0, 114.0]]
[[50.0, 70.0]]
[[93.0, 99.0]]
[[129.0, 134.0]]
[[88.0, 94.0]]
[[63.0, 73.0]]
[[103.0, 118.0]]
[[91.0, 95.0]]
[[91.0, 97.0]]
[[73.0, 78.0], [83.0, 88.0], 

In [0]:
print(len(new_train_gb))
print(len(new_extraction))

1080
1431


In [0]:
final_train

[(['do',
   'you',
   'have',
   'any',
   'medication',
   'allergy',
   'asthma',
   'me',
   'pt',
   'no',
   'wait',
   'avelox',
   'thats',
   'it',
   'so',
   'no',
   'other',
   'allergy',
   'right',
   'cont'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'U-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['if',
   'avelox',
   'ha',
   'hurt',
   'your',
   'liver',
   'avoid',
   'tylenol',
   'always',
   'a',
   'it',
   'further',
   'damage',
   'liver',
   'eat',
   'grapefruit',
   'unless',
   'taking',
   'cardiac',
   'drug'],
  ['O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'L-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['apparently',
   'baclofen',
   'greatly',
   'exacerbates',
   'the',
   'ad',
   'part',
   'of',
   'my',
   'adhd',
   'average',
   'length',
   'of',
   'focus',
   'today',
   'about',
  

In [0]:
import pickle

with open('/content/gdrive/My Drive/Task2_data/test.pkl', 'wb') as f:
  pickle.dump(final_train, f)

In [0]:
len(final_train)

560

In [0]:
from biobert_embedding.embedding import BiobertEmbedding
biobert = BiobertEmbedding()

In [0]:
biobert(["dasd"])

TypeError: ignored